In [ ]:
# Treatment train - RO (RPT1) 
    # Power consumption vs recovery
# PV surrogate model
# Show PV+RO results :  We want higher recovery
# RPT-ZLD
    # Show functions adding MD and MEC to flowsheet
    # MD has high thermal energy requirements (show plots # agg heat and electricity sweep)
    # Build CST surrogate
    # Present the results % solar energy sweep

In [ ]:
# Relevant Imports
from IPython.display import Image,HTML
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', { 'width':1400, 'height': 768, 'scroll': True, })

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from watertap.core.util.model_diagnostics.infeasible import *
from watertap.core.util.initialization import *

from pyomo.environ import (
    ConcreteModel,
    assert_optimal_termination
)
from pyomo.util.calc_var_value import calculate_variable_from_constraint as cvc

from idaes.core import FlowsheetBlock, UnitModelCostingBlock
from idaes.core.util.model_statistics import *

from watertap.core.solvers import get_solver

from watertap_contrib.reflo.code_demos.REFLO_demo.flowsheets.sweep_functions.results_dict import *
from pyomo.environ import (
    value,
)

from idaes.core.util.model_statistics import degrees_of_freedom

from IPython.display import clear_output
from watertap_contrib.reflo.analysis.case_studies.KBHDP import *

plt.rcParams["axes.labelsize"] = 12
plt.rcParams["axes.titlesize"] = 14

cwd = %pwd

solver = get_solver()

In [ ]:
def pv_plots(m):
    rd = build_results_dict(m)

    electricity_annual_model = list()
    electricity_annual_error = list()
    electricity_annual_error_rel = list()

    land_req_model = list()
    land_req_error = list()
    land_req_error_rel = list()

    for row in m.fs.pv.data.iterrows():
        m.fs.pv.design_size.fix(row[1].design_size)

        results = solver.solve(m, tee=False)
        assert_optimal_termination(results)
        electricity_annual_model.append(value(m.fs.pv.annual_energy))
        electricity_annual_error.append(value(m.fs.pv.annual_energy) - row[1].annual_energy)
        electricity_annual_error_rel.append(
            100
            * (value(m.fs.pv.annual_energy) - row[1].annual_energy)
            / row[1].annual_energy
        )
        land_req_model.append(value(m.fs.pv.land_req))
        land_req_error.append(value(m.fs.pv.land_req) - row[1].land_req)
        land_req_error_rel.append(
            100 * (value(m.fs.pv.land_req) - row[1].land_req) / row[1].land_req
        )
    #     rd = results_dict_append(m, rd)
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(9, 7))
    axs[0][0].scatter(
        m.fs.pv.data.annual_energy,
        electricity_annual_model,
        marker="x",
    )
    axs[0][0].plot(
        [0, max(m.fs.pv.data.annual_energy)],
        [0, max(m.fs.pv.data.annual_energy)],
        color="black",
        linestyle="--",
    )
    axs[0][0].set(
        xlabel="Electrical Energy Generated (Data)",
        ylabel="Electrical Energy Generated (REFLO)",
    )

    axs[0][1].scatter(
        m.fs.pv.data.annual_energy, electricity_annual_error_rel, marker="x", color="k"
    )
    axs[0][1].plot(
        m.fs.pv.data.annual_energy,
        [0 for _ in m.fs.pv.data.annual_energy],
        color="black",
        linestyle="--",
    )
    axs[0][1].set(xlabel="Electrical Energy Generated (Data)", ylabel="% Error")

    axs[1][0].scatter(
        m.fs.pv.data.land_req,
        land_req_model,
        color="red",
        marker="x",
    )
    axs[1][0].plot(
        [0, max(m.fs.pv.data.land_req)],
        [0, max(m.fs.pv.data.land_req)],
        color="black",
        linestyle="--",
    )
    axs[1][0].set(xlabel="Land Required (Data)", ylabel="Land Required (WaterTAP)")

    axs[1][1].scatter(m.fs.pv.data.land_req, land_req_error_rel, marker="x", color="k")
    axs[1][1].plot(
        m.fs.pv.data.land_req,
        [0 for _ in m.fs.pv.data.land_req],
        color="black",
        linestyle="--",
    )
    axs[1][1].set(xlabel="Land Required (Data)", ylabel="% Error")
    fig.tight_layout()

In [ ]:
def cst_plots(m):
    heat_annual_model = list()
    heat_annual_error = list()
    heat_annual_error_rel = list()

    electricity_annual_model = list()
    electricity_annual_error = list()
    electricity_annual_error_rel = list()

    for row in m.fs.cst.data.iterrows():

        m.fs.cst.heat_load.fix(row[1].heat_load)
        m.fs.cst.hours_storage.fix(row[1].hours_storage)
        m.fs.cst.initialize()

        try:
            results = solver.solve(m, tee=False)
        except:
            print_infeasible_constraints(m)
            assert False
        assert_optimal_termination(results)
        heat_annual_model.append(value(m.fs.cst.heat_annual))
        electricity_annual_model.append(value(m.fs.cst.electricity_annual))
        heat_annual_error.append(value(m.fs.cst.heat_annual) - row[1].heat_annual)
        electricity_annual_error.append(
            value(m.fs.cst.electricity_annual) - row[1].electricity_annual
        )
        heat_annual_error_rel.append(
            100 * (value(m.fs.cst.heat_annual) - row[1].heat_annual) / row[1].heat_annual
        )
        electricity_annual_error_rel.append(
            100
            * (value(m.fs.cst.electricity_annual) - row[1].electricity_annual)
            / row[1].electricity_annual
        )
#         rd = results_dict_append(m, rd)

    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(9, 7))
    axs[0][0].scatter(
        m.fs.cst.data.heat_annual,
        heat_annual_model,
        label="Heat Annual Model",
        marker="x",
    )
    axs[0][0].plot(
        [0, max(m.fs.cst.data.heat_annual)],
        [0, max(m.fs.cst.data.heat_annual)],
        color="black",
        linestyle="--",
    )
    axs[0][0].set(
        xlabel="Thermal Energy Generated (Data)",
        ylabel="Thermal Energy Generated (WaterTAP)",
    )

    axs[0][1].scatter(
        m.fs.cst.data.heat_annual, heat_annual_error_rel, marker="x", color="k"
    )
    axs[0][1].plot(
        m.fs.cst.data.heat_annual,
        [0 for _ in m.fs.cst.data.heat_annual],
        color="black",
        linestyle="--",
    )
    axs[0][1].set(xlabel="Annual Heat Energy (Data)", ylabel="% Error")
    axs[1][0].scatter(
        m.fs.cst.data.electricity_annual,
        electricity_annual_model,
        color="red",
        marker="x",
    )
    axs[1][0].plot(
        [0, max(m.fs.cst.data.electricity_annual)],
        [0, max(m.fs.cst.data.electricity_annual)],
        color="black",
        linestyle="--",
    )
    axs[1][0].set(
        xlabel="Electric Power Consumed (Data)", ylabel="Electric Power Consumed (WaterTAP)"
    )

    axs[1][1].scatter(
        m.fs.cst.data.electricity_annual,
        heat_annual_error_rel,
        label="Electricity Annual Model",
        marker="x",
        color="k",
    )
    axs[1][1].plot(
        m.fs.cst.data.electricity_annual,
        [0 for _ in m.fs.cst.data.electricity_annual],
        label="Electricity Annual Model",
        color="black",
        linestyle="--",
    )
    axs[1][1].set(xlabel="Electric Power Consumed (Data)", ylabel="% Error")
    fig.tight_layout()

Intro
- basic WaterTAP stuff
- touch on the compoenents of WaterTAP?
 
Figure of RPT1 flowsheet
 
RPT1
 
RPT1 - recovery sweep; get power demand
 
RPT1 - build PV surrogate based on power demand and RE fraction desired
 
Break for PV surrogate
 
Show Summary of this system
- recovery fraction, and final brine concentration
- cost metrics (LCOW, SEC)
- energy costs
- stacked plot
 
Figure of MLD flowsheet
 
80% recovery of RO
 
MLD - MD recovery sweep; get thermal power demand
 
MLD - build CST surrogate based on power demand
 
Break for CST surrogate
 
Show Summary of this system
- recovery fraction, and final brine concentration
- cost metrics (LCOW, SEC)
- energy costs
- stacked plot
 
Repeat for ZLD
 
Break for CST surrogate
 
Final Product would be summary of the three treatment trains and comparison

<h1>Demonstration of WaterTAP-REFLO Framework</h1>

<div 
    style=
     "max-height:800px; 
      overflow-y: auto;">
    
Contents:
<li>WaterTAP-REFLO framework</li>
<li>Treatment flowsheet workflow in WaterTAP-REFLO</li>
<ul>
    <li>Build, optimization and costing with a single unit model
    <li>Parameter sweeps and results
</ul>
    
<li>Build example analysis flowsheet energy</li>
<ul>
    <li>Build a complete treatment flowsheet
    <li>Build solar surrogate    
    <li>Integrate solar energy models
    <li>Parameter sweeps and results  
</ul>
<li>Comparative Analysis</li>
</div>

WaterTAP-REFLO Framework

<center><img src="figures/slides/slide14.jpeg" width=1000 height=1000>

The REFLOSystem Costing Package essential balances energy demand of the Water Treatment System with grid or renewable energy technologies
<center><img src="figures/slides/slide4.jpeg" width=1000 height=1000>

The levelized cost of treatment (LCOT) is determined by <br>
optimizing the water treatment system size and using the Treatment Costing package
<img src="figures/slides/slide1.jpeg" width=1200 height=1200>

The Energy System is sized based on the electricity/heat demand
and costed <br> using the Energy Costing package
<img src="figures/slides/slide2.jpeg" width=1200 height=1200>

The REFLOSystem Costing package connects the energy demand
of the Water Treatment System to the Energy System 
to optimize the for a LCOW with energy integration
<center><img src="figures/slides/slide3.jpeg" width=1000 height=1000>

Treatment flowsheet workflow in WaterTAP-REFLO

Water treatment train to be modeled
<left><img src="figures/slides/slide6.jpeg" width=1000 height=1000><left>

1. Import relevant libraries

Build an analysis flowsheet with energy integration

<center><img src="figures/slides/slide7.jpeg" width=1000 height=1000><center>

Code Snippet

In [ ]:
# Treatment train - RO (RPT1) 
    # Power consumption vs recovery

from watertap_contrib.reflo.code_demos.REFLO_demo.flowsheets.demo_kbhdp_ro import *

def demo1_grid_only(
        ro_recovery=0.8,
        heat_price=0.0,
        electricity_price=0.04989,
        dwi_lcow=0.58,
    ):
    '''This is modified for demonstration purposes'''
    
    #### Add a concrete flowsheet
    m = build_demo1_system()

    #### Add pretreatment and RO to the flowsheet. 
    #### This includes all the build, connections, initialization and solve
    m = build_kbhdp_ro(m,ro_recovery)

    #### Add product and deep well injection
    add_product(m)
    
    #### Treatment costing block
    add_treatment_costing(m,heat_price,electricity_price)
    
    #### Update DWI costing
    m.fs.treatment.costing.deep_well_injection.dwi_lcow.set_value(dwi_lcow)
    
    #### Set objective function
    m.fs.lcow_objective = Objective(expr=m.fs.treatment.costing.LCOW)

    results = solve(m)
    return m


RO Recovery Sweep

<img src="figures/results/kbhdp-ro-recovery-sweep.png" width=45% style="display:inline-block;"/> 
<img src="figures/results/kbhdp-ro-energy.png" width=45% style="display:inline-block;"/>

PySAM surrogates are built to meet the annual energy consumption of treatment system

<center><img src="figures/slides/slide5.jpeg" width=50% style="display:inline-block;"/> 
    
1. Weather file
2. Energy demand range of treatment system
3. Determine the inputs to PySAM model. Eg. System capacity, hours of storage, operating temperature etc.
4. Run PySAM for range of input
5. Create surrogate using outputs such as annual heat/electricity from PySAM in WaterTAP-REFLO

PV Surrogate: Users can decide the input to the surrogate model

<center><img src="figures/slides/slide13.jpeg" width="80%" style="display:inline-block;"/><center>

In [ ]:
# Generate data

import numpy as np
from watertap_contrib.reflo.solar_models.surrogate import generate_pv_data

weather_file = "/Users/ksitterl/Documents/Python/watertap-reflo/watertap-reflo/src/watertap_contrib/reflo/flowsheets/KBHDP/data/el_paso_texas-KBHDP-weather.csv"  # Update with path to weather file

pv_data = generate_pv_data(
    system_capacities=np.linspace(100, 2000, 100),
    pysam_model_config="FlatPlatePVSingleOwner",
    save_data=True,
    use_multiprocessing=True,
    processes=8,
    dataset_filename="pv_surrogate_data.pkl",
    weather_file=weather_file,
    tech_config_file=None,
    grid_config_file=None,
    rate_config_file=None,
    cash_config_file=None,
)
clear_output(wait=False)

In [ ]:
# PV Surrogate

from watertap_contrib.reflo.solar_models.surrogate.pv import PVSurrogate
input_variables = {
    "labels": ["design_size"],
    "bounds": {"design_size": [1000, 10000]},
    "units": {"design_size": "kW"},
}
output_variables = {
    "labels": ["annual_energy", "land_req"],
    "units": {"annual_energy": "kWh", "land_req": "acre"},
}
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.costing = EnergyCosting()
m.fs.pv = PVSurrogate(
    dataset_filename="data/demo_pv.pkl",
    input_variables=input_variables,
    output_variables=output_variables,
    scale_training_data=False,
)
clear_output(wait=False)

In [ ]:
pv_plots(m)

Code Snippet with Integration of Solar Energy (PV)

In [ ]:
# Update code snippet to show adding PV to the RO flowsheet


Sweep across RO water recovery and Solar energy integration

<img src="figures/results/kbhdp-ro-solar-recovery-sweep.png" width=45% style="display:inline-block;"/>
<img src="figures/results/kbhdp-ro-solar-grid-sweep.png" width=45% style="display:inline-block;"/>

A sweep across DWI costs highlights its relative importance to other costs

<center><img src="figures/results/kbhdp-ro-dwi-sweep-2.png" width="50%"/> 

In [ ]:
# CST surrogate
# Users can decide the input to the surrogate model-300C
# Net metering and 24h storage of thermal system
# Accuracy of surrogate

CST Surrogate: Users can decide the input to the surrogate model

<center><img src="figures/slides/slide12.jpeg" width="80%" style="display:inline-block;"/><center>

In [ ]:
# Trough surrogate model
from watertap_contrib.reflo.solar_models.surrogate.trough.trough_surrogate import (
    TroughSurrogate,
)

input_bounds = dict(heat_load=[5, 25], hours_storage=[12, 24])
input_units = dict(heat_load="MW", hours_storage="hour")
input_variables = {
    "labels": ["heat_load", "hours_storage"],
    "bounds": input_bounds,
    "units": input_units,
}

output_units = dict(
    heat_annual_scaled="kWh",
    electricity_annual_scaled="kWh",
    total_aperture_area_scaled="m**2",
)
output_variables = {
    "labels": [
        "heat_annual_scaled",
        "electricity_annual_scaled",
        "total_aperture_area_scaled",
    ],
    "units": output_units,
}
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.cst = TroughSurrogate(
    dataset_filename="data/trough_demo_mld.pkl",
    input_variables=input_variables,
    output_variables=output_variables,
    scale_training_data=True,)
#
clear_output(wait=False)

In [ ]:
cst_plots(m)

Zero Liquid Discharge - Brine management step updated to crystallizer

<center><img src="figures/slides/slide8.jpeg" width=1000 height=1000><center>

Sweep across MD water recovery

<img src="figures/results/kbhdp-mld-elec.png" width="32%" style="display:inline-block;"/>
<img src="figures/results/kbhdp-mld-heat.png" width="32%" style="display:inline-block;"/>

Sweep across solar integration fraction

<img src="figures/results/kbhdp-zld-solar-grid-sweep.png" width="35%" style="display:inline-block;">
<img src="figures/results/kbhdp_zld_tornado_plots.png" width="55%" style="display:inline-block;">

Future Capabilities

Multiperiod PV-RO

<center><img src="figures/slides/slide10.jpeg" width="80%" style="display:inline-block;">

Multiperiod Thermal System

<center><img src="figures/slides/slide11.jpeg" width="90%" style="display:inline-block;">

1. Location specific remote treatment system optimization - Datacenters
2. Permian - brackish brine management
3. Flexible desalination - NAWI not very focused on renewable energy yet - Pricetaker model uses multiperiod framework and grid prices